In [ ]:
pip install pennylane torch scikit-learn

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import torch
from torch import nn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
# Load the dataset
file_path = 'C:\\Users\\Nirusan03\\PycharmProjects\\FYP_POC\\Final_Dataset.csv'  # Update path
dataset = pd.read_csv(file_path)

# Selecting features and label
features = dataset.drop(columns=['Label', 'Label.1'])[:100]  # Use only a subset to reduce computation
label = dataset['Label'][:100]

# Encode and scale the data
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(label)

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features_scaled, encoded_labels, test_size=0.2, random_state=42)

In [ ]:
# Define a 2-qubit quantum device
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

# Quantum circuit to process input data
def quantum_circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [ ]:
# Quantum node
n_layers = 2
weight_shapes = {"weights": (n_layers, n_qubits)}
qnode = qml.QNode(quantum_circuit, dev, interface="torch")

# Define the hybrid quantum-classical layer
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.q_params = nn.Parameter(torch.randn(weight_shapes["weights"], dtype=torch.float32))  # Ensure float32
        self.fc1 = nn.Linear(features_scaled.shape[1], n_qubits)
        
        # Adjust fc2's input size based on actual output from qnode
        self.fc2 = nn.Linear(n_qubits, 1)  # Adjust if needed after confirming q_out size

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        
        # Execute the quantum node
        q_out = qnode(x, self.q_params)
        
        # Convert q_out to a tensor and stack if it is a list
        if isinstance(q_out, list):
            q_out = torch.stack([torch.tensor(val, dtype=torch.float32) for val in q_out])
        
        # Ensure q_out shape compatibility
        q_out = q_out.view(-1, n_qubits)  # Reshape q_out to have compatible shape for fc2
        
        # Pass through the final fully connected layer
        x = self.fc2(q_out)
        return torch.sigmoid(x)  # Output probability for binary classification


In [ ]:
# Initialize the model, loss, and optimizer
model = HybridModel()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Data Preparation Section

# Check that target values are in the range [0, 1]
unique_labels = set(y_train)
print("Unique values in y_train:", unique_labels)

# Convert to PyTorch tensors with correct data types
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# Convert y_train to binary by mapping all values > 1 to 1
y_train = [0 if label == 0 else 1 for label in y_train]
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
print("Updated unique values in y_train:", set(y_train))

# Confirm target tensor is in the correct format for BCELoss
print("Target tensor dtype and shape:", y_train_tensor.dtype, y_train_tensor.shape)

In [ ]:
# Test model with a single sample to check for compatibility
optimizer.zero_grad()
output = model(X_train_tensor[:1])  # Forward pass with a single sample
sample_loss = criterion(output, y_train_tensor[:1])  # Calculate sample loss
print("Sample loss calculation successful:", sample_loss.item())

In [ ]:
n_epochs = 10
for epoch in range(n_epochs):
    optimizer.zero_grad()
    output = model(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set model to evaluation mode
model.eval()
with torch.no_grad():
    y_pred_probs = model(X_test_tensor)
    y_pred = y_pred_probs.argmax(dim=1)  # For multiclass, take the class with highest probability

# Convert tensors to numpy arrays for compatibility
y_pred_np = y_pred.cpu().numpy()
y_test_np = y_test_tensor.cpu().numpy()

# Calculate evaluation metrics for multiclass classification
accuracy = accuracy_score(y_test_np, y_pred_np)
precision = precision_score(y_test_np, y_pred_np, average='macro', zero_division=1)
recall = recall_score(y_test_np, y_pred_np, average='macro', zero_division=1)
f1 = f1_score(y_test_np, y_pred_np, average='macro', zero_division=1)

# Print evaluation results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (macro): {precision:.4f}")
print(f"Recall (macro): {recall:.4f}")
print(f"F1 Score (macro): {f1:.4f}")